## Protocol Steps
1. Alice and Bob create n EPR pairs.
2. For each pair, Alice and Bob measure randomly from the {a1,a2,a3} and {b1,b2,b3} basis respectivly. Each of these bases is a rotation around the bloch sphere where a2=b1 and a3=b2
3. Alice and Bob share their measurement bases over a public channel 
4. Alice and Bob both remoove all the bits in their strings corresponding to 0s in the string bob just sent


In [70]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
import pandas as pd
from math import pi, sqrt

### Set meta variables

In [71]:
n = 300
test = True
rand_seed = False

eve = False

if rand_seed:
    np.random.seed(seed=0)

### This function simple takes a number and returns a list of that length filled with EPR pairs

In [72]:
def create_EPR_pairs(n):
    pairs = []
    for i in range(n):
        qc = QuantumCircuit(2,2)
        qc.h(0)
        qc.cx(0,1)
        pairs.append(qc)
    return pairs

### This function measures a single entangled bit with two different bases and returns their result in a tuple

In [73]:
# Measures the two bits of an EPR pair with a basis that is the z basis rotated by the given angle
def measure_ang(qca, a_base, b_base):
    # First measure alices bit and then measure bobs

    if a_base == 0:
        # Measure directly in the z basis
        qca.measure(0,0)

    elif a_base == 1:
        # rotate the y basis by pi/4
        qca.u(pi/4,0,0,0)
        qca.measure(0,0)
        qca.u(-pi/4,0,0,0)
        
    elif a_base == 2:
        # rotate the y basis by pi/2
        qca.u(pi/2,0,0,0)
        qca.measure(0,0)
        qca.u(-pi/2,0,0,0)
        
    if b_base == 0:
        # rotate the z basis by pi/4
        qca.u(pi/4,0,0,1)
        qca.measure(1,1)
        qca.u(-pi/4,0,0,1)
        
    elif b_base == 1:
        # rotate the z basis by pi/2
        qca.u(pi/2,0,0,1)
        qca.measure(1,1)
        qca.u(-pi/2,0,0,1)
        
    elif b_base == 2:
        # rotate the z basis by 3pi/4
        qca.u(3*pi/4,0,0,1)
        qca.measure(1,1)
        qca.u(-3*pi/4,0,0,1)
    
    # Simulate the measurement (taken directly from the qiskit example)
    qasm_sim = Aer.get_backend('qasm_simulator')
    qobj = assemble(qca, shots=1, memory=True)
    result = list(qasm_sim.run(qobj).result().get_memory()[0])    
    return int(result[0]),int(result[1])
    

def measure(pairs, alice_bases, bob_bases):
    alice_outcome, bob_outcome = [],[]
    for i, pair in enumerate(pairs):
        # If measuring in compatible bases, measure and if not add a dummy value of 2
        a_out, b_out = measure_ang(pair, alice_bases[i], bob_bases[i])
        alice_outcome.append(a_out)
        bob_outcome.append(b_out)
    return alice_outcome, bob_outcome
        

### This function takes alice, bobs and eves measurements and bases and returns the final keys and the CHSH test value 

In [74]:
def trash(a,b,a_bases, b_bases, eve_a, eve_b):
    alice_key = []
    bob_key = []
    eve_key = []
    a1b1 = []
    a1b3 = []
    a3b1 = []
    a3b3 = []

    for i in range(len(a_bases)):
        if (a_bases[i] == 1 and b_bases[i] == 0) or (a_bases[i] == 2 and b_bases[i] == 1):
            alice_key.append(a[i])
            bob_key.append(b[i])
            if eve:
                eve_key.append(eve_a[i])
        elif a_bases[i] == 0 and b_bases[i] == 0:
            a1b1.append((a[i],b[i]))
        elif a_bases[i] == 0 and b_bases[i] == 2:
            a1b3.append((a[i],b[i]))
        elif a_bases[i] == 2 and b_bases[i] == 0:
            a3b1.append((a[i],b[i]))
        elif a_bases[i] == 2 and b_bases[i] == 2:
            a3b3.append((a[i],b[i]))
            
            
        E = []
        for outcome in [a1b1, a1b3, a3b1, a3b3]:
            tot = 0 
            count = 0

            for result in outcome:
                if (result == (1,1)) or (result == (0,0)):
                    count += 1
                else:
                    count -= 1
                tot += 1
                
            if tot != 0:
                E.append(count/tot)
            else:
                E.append(0)
                
        S = E[0] - E[1] + E[2] + E[3]
    return alice_key, bob_key, eve_key, S

### This function takes a list of EPR pairs and measures all the qubits in the z basis

In [75]:
def eve_intercept(EPR_pairs):
    bob_result = []
    alice_result = []

    for pair in EPR_pairs:
        pair.measure(0,0)
        pair.measure(1,1)

        qasm_sim = Aer.get_backend('qasm_simulator')
        qobj = assemble(pair, shots=1, memory=True)
        result = list(qasm_sim.run(qobj).result().get_memory()[0])   
        alice_result.append(int(result[0]))
        bob_result.append(int(result[1])) 
    return alice_result,bob_result


## First trial
We can first run with no Eve and an n of 300. We would expect Alice and Bob to get matching keys. We would also expect the CHSH test to yield around 2 root 2.


In [76]:
n = 300 
eve = False

First we will create our n EPR pairs and generate random bases for both Alice and Bob

In [77]:
alice_bases = randint(3, size=n)
bob_bases = randint(3, size=n)

EPR_pairs = create_EPR_pairs(n)

Next Eve and Bob will both measure the entangled bits in their respective bases

In [78]:
a,b = measure(EPR_pairs, alice_bases, bob_bases)

Finally Alice and Bob will communicate over a public channel their bases for measurement. All bits that match in measurement basis will be used for the final key. All bits that had different measurements will be used for the CHSH test.

In [79]:
alice_key, bob_key, eve_key, S = trash(a,b,alice_bases,bob_bases, None, None)

if alice_key == bob_key:
    print(f'Alice and Bob have matching keys of length {len(alice_key)}. The key is below')
    print(alice_key, end='\n\n')

    if S > 2:
        print(f'The CHSH test has been passed with a value of {round(S,3)} and Alice and Bob can safely use this key.')
    else:
        print('The CHSH test has not been passed and this key cannot be used safely.')

else:
    print('Alice and Bob do not have matching keys! Something has gone wrong in the protocol.')

Alice and Bob have matching keys of length 72. The key is below
[0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0]

The CHSH test has been passed with a value of 2.757 and Alice and Bob can safely use this key.


## Second trial
We will now introduce Eve to the system. We will run the test with an n of 300 again.

In [80]:
n = 300 
eve = True

We will generate EPR pairs and bases in the same way as above.

In [81]:
alice_bases = randint(3, size=n)
bob_bases = randint(3, size=n)

EPR_pairs = create_EPR_pairs(n)

Now eve will intercept and measure the pairs sent on the public channel

In [82]:
eve_a, eve_b = eve_intercept(EPR_pairs)

Finally Alice and Bob measure their qubits and compare their results

In [85]:
a,b = measure(EPR_pairs, alice_bases, bob_bases)
alice_key, bob_key, eve_key, S = trash(a,b,alice_bases,bob_bases, eve_a, eve_b)

if S < 2:
    print(f'The CHSH test has yielded a value of {round(S,3)} which is below 2, so Eve was detected on the network and this key is not safe to use.')
else:
    print('By chance, this trial has passed the CHSH test and Eve went undetected.')



The CHSH test has yielded a value of 1.767 which is below 2, so Eve was detected on the network and this key is not safe to use.
